<div class="mark">
Part I: FEATURE ENGINEERING AND MODEL PREPARATION</div><i class="fa fa-lightbulb-o "></i>

In [1]:
# STEP 1- importing packages and loading data.

# importing libraries
import pandas as pd # data science essentials
import matplotlib.pyplot as plt # essential graphical output
import seaborn as sns # enhanced graphical output
import numpy as np
import statsmodels.formula.api as smf

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# specifying file name
file = './Apprentice_Chef_Dataset.xlsx'


# reading the file into Python
chef = pd.read_excel(io=file)

In [2]:
#STEP 2, feature engineering emails.

#Using EMAIL to get the domain out and then catogorize the email domains
# step a

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in chef.iterrows():
    
    # splitting email domain at '@'
    split_email = chef.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# displaying the results
email_df

,0,1
0,saathos,unitedhealth.com
1,alysanne.osgrey,ge.org
2,edwyd.fossoway,jnj.com
3,eleyna.westerling,ge.org
4,elyn.norridge,jnj.com
...,...,...
1941,obara.sand,yahoo.com
1942,quentyn.blackwood,yahoo.com
1943,rhonda.rowan,gmail.com
1944,turnip,yahoo.com


In [3]:
# step b: remove ".com" from domains
# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in email_df.iterrows():
    
    
    replace_email = email_df.iloc[index, 1].replace('.com','')

    placeholder_lst.append(replace_email)
    
email_df = pd.DataFrame(placeholder_lst)


# displaying the results
email_df

,0
0,unitedhealth
1,ge.org
2,jnj
3,ge.org
4,jnj
...,...
1941,yahoo
1942,yahoo
1943,gmail
1944,yahoo


In [4]:
#step c, removing ".org" in domains
# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in email_df.iterrows():
    
    
    replace_email = email_df.iloc[index, 0].replace('.org','')
  
    placeholder_lst.append(replace_email)
    
email_df = pd.DataFrame(placeholder_lst)


# displaying the results

email_df
email_df['EMAIL_domain']=email_df.iloc[:,0]
email_df.head()

,0,EMAIL_domain
0,unitedhealth,unitedhealth
1,ge,ge
2,jnj,jnj
3,ge,ge
4,jnj,jnj


In [5]:
# STEP d: concatenating with original DataFrame

# safety measure in case of multiple concatenations
chef = pd.read_excel(file)


# renaming column to concatenate
email_df.columns = ["0" , "EMAIL_domain"]


# concatenating personal_email_domain with friends DataFrame
chef = pd.concat([chef, email_df["EMAIL_domain"]],
                   axis = 1)


# printing value counts of personal_email_domain
#chef.loc[: ,'EMAIL_domain'].value_counts()

In [6]:
# STEP e: Categorize email domains to personal and business
personal_email_domains = ['gmail', 'protonmail','yahoo','msn',
                         'aol','passport','hotmail','live','me']
business_email_domains  = ['homedepot','intel','unitedtech','cisco','goldmansacs',
                              'jpmorgan','visa','pfizer','disney','walmart','unitedhealth',
'boeing','caterpillar','verizon','pg','dupont','ibm','chevron','microsoft', 
'travelers','exxon','amex','cocacola','mcdonalds','merck','jnj','apple','nike',          
'ge','mmm']

# placeholder list
placeholder_lst = []


# looping to group observations by domain type
for domain in chef['EMAIL_domain']:
    
    if domain in personal_email_domains:
        placeholder_lst.append('personal')
        

    elif domain in business_email_domains:
        placeholder_lst.append('business')


    else:
            print('Unknown')


# concatenating with original DataFrame
chef['DOMAIN_group'] = pd.Series(placeholder_lst)


# checking results
chef['DOMAIN_group'].value_counts()

personal    1250
business     696
Name: DOMAIN_group, dtype: int64

In [7]:
# STEP f, creating "EMAIL_type" column, # 1 for personal email, 0 for business.
chef['EMAIL_type']   = 0



for index, value in chef.iterrows():
    
   
    if chef.loc[index, 'DOMAIN_group'] == 'business':
        chef.loc[index, 'EMAIL_type'] = 0

    else: 
        chef.loc[index, 'EMAIL_type'] = 1
        


In [8]:
# STEP g, Getting dummy variables from catogory variable"EMAIL_domain"
# one hot encoding categorical variables
one_hot_email       = pd.get_dummies(chef['EMAIL_domain'])





# joining codings together
chef = chef.join([one_hot_email])

In [9]:
# dropping "NAME","EMAIL", "FIRST_NAME", "FAMILY_NAME","EMAIL_DOMAIN", AND
#"DOMAIN_group" from model development.

chef = chef.drop('NAME', axis = 1) # can't run more than one time
chef = chef.drop('EMAIL', axis = 1)
chef = chef.drop('FIRST_NAME', axis = 1)
chef= chef.drop('FAMILY_NAME', axis = 1)
chef = chef.drop('EMAIL_domain', axis = 1)
chef = chef.drop('DOMAIN_group', axis = 1)


In [10]:
# renaming misleading column and create a "log_AVG_ORDER_SIZE"
chef=chef.rename({'LARGEST_ORDER_SIZE':'AVG_ORDER_SIZE'}, axis=1)
chef["log_AVG_ORDER_SIZE"]=np.log10(chef.loc[:,"AVG_ORDER_SIZE"])

In [11]:
# STEP 3, log transforming

chef['log_REVENUE'] = np.log10(chef['REVENUE'])
chef['log_AVG_TIME_PER_SITE_VISIT'] = np.log10(chef['AVG_TIME_PER_SITE_VISIT'])
chef['log_AVG_PREP_VID_TIME']=np.log10(chef['AVG_PREP_VID_TIME'])
chef['log_TOTAL_MEALS_ORDERED'] = np.log10(chef['TOTAL_MEALS_ORDERED'])
chef['log_UNIQUE_MEALS_PURCH'] = np.log10(chef.loc[:, 'UNIQUE_MEALS_PURCH'])
chef["log_CONTACTS_W_CUSTOMER_SERVICE"]=np.log10(chef.loc[:,"CONTACTS_W_CUSTOMER_SERVICE"])
chef['log_PRODUCT_CATEGORIES_VIEWED']=np.log10(chef.loc[:,"PRODUCT_CATEGORIES_VIEWED"])
chef["log_AVG_CLICKS_PER_VISIT"]=np.log10(chef.loc[:,"AVG_CLICKS_PER_VISIT"])

In [12]:
# STEP 4, creating binary if counts of 0s is greater than 100.
chef['has_WEEKLY_PLAN']   = 0
chef['has_VIEWED_PHOTOS'] = 0
chef['has_EARLY_DELIVERIES'] = 0
chef['has_LATE_DELIVERIES'] = 0
chef['has_MASTER_CLASSES_ATTENDED'] = 0

for index, value in chef.iterrows():
    
    # Total_Bsmt_SF
    if chef.loc[index, 'WEEKLY_PLAN'] > 0:
        chef.loc[index, 'has_WEEKLY_PLAN'] = 1


    if chef.loc[index, 'TOTAL_PHOTOS_VIEWED'] > 0:
        chef.loc[index, 'has_VIEWED_PHOTOS'] = 1
            # Total_Bsmt_SF
    if chef.loc[index, 'EARLY_DELIVERIES'] > 0:
        chef.loc[index, 'has_EARLY_DELIVERIES'] = 1


    if chef.loc[index, 'LATE_DELIVERIES'] > 0:
        chef.loc[index, 'has_LATE_DELIVERIES'] = 1
        
    if chef.loc[index, 'MASTER_CLASSES_ATTENDED'] > 0:
        chef.loc[index, 'has_MASTER_CLASSES_ATTENDED'] = 1   
        

In [13]:
# STEP6,
#making a copy of chef
chef_explanatory = chef.copy()


# dropping REVENUE,log_REVENUE,NAME, FIRST_NAME, FAMILY_NAME, and EMAIL from the explanatory variable set
chef_explanatory = chef_explanatory.drop(['REVENUE',
                                        'log_REVENUE'], axis = 1)


# formatting each explanatory variable for statsmodels
for val in chef_explanatory:
    print(f"{val} +")

CROSS_SELL_SUCCESS +
TOTAL_MEALS_ORDERED +
UNIQUE_MEALS_PURCH +
CONTACTS_W_CUSTOMER_SERVICE +
PRODUCT_CATEGORIES_VIEWED +
AVG_TIME_PER_SITE_VISIT +
MOBILE_NUMBER +
CANCELLATIONS_BEFORE_NOON +
CANCELLATIONS_AFTER_NOON +
TASTES_AND_PREFERENCES +
PC_LOGINS +
MOBILE_LOGINS +
WEEKLY_PLAN +
EARLY_DELIVERIES +
LATE_DELIVERIES +
PACKAGE_LOCKER +
REFRIGERATED_LOCKER +
AVG_PREP_VID_TIME +
AVG_ORDER_SIZE +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
AVG_CLICKS_PER_VISIT +
TOTAL_PHOTOS_VIEWED +
EMAIL_type +
amex +
aol +
apple +
boeing +
caterpillar +
chevron +
cisco +
cocacola +
disney +
dupont +
exxon +
ge +
gmail +
goldmansacs +
homedepot +
hotmail +
ibm +
intel +
jnj +
jpmorgan +
live +
mcdonalds +
me +
merck +
microsoft +
mmm +
msn +
nike +
passport +
pfizer +
pg +
protonmail +
travelers +
unitedhealth +
unitedtech +
verizon +
visa +
walmart +
yahoo +
log_AVG_ORDER_SIZE +
log_AVG_TIME_PER_SITE_VISIT +
log_AVG_PREP_VID_TIME +
log_TOTAL_MEALS_ORDERED +
log_UNIQUE_MEALS_PURCH +
log_CONTACTS_W_

In [14]:
# STEP 7, 
# saving engineered dataset in Excel
chef.to_excel('./chef_ready.xlsx',
                 index = False)

In [15]:
# STEP 8-1,building a full model WITH log_REVENUE

# blueprinting a model type
lm_full = smf.ols(formula = """log_REVENUE ~ CROSS_SELL_SUCCESS +
PRODUCT_CATEGORIES_VIEWED +
TASTES_AND_PREFERENCES +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
TOTAL_PHOTOS_VIEWED +
yahoo +
log_AVG_ORDER_SIZE +
log_AVG_PREP_VID_TIME +
log_TOTAL_MEALS_ORDERED +
log_UNIQUE_MEALS_PURCH +
log_CONTACTS_W_CUSTOMER_SERVICE +
has_WEEKLY_PLAN +
has_VIEWED_PHOTOS 
""",
                               data = chef)


# telling Python to run the data through the blueprint
results_full = lm_full.fit()


# printing the results
results_full.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_REVENUE   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     371.5
Date:                Mon, 05 Jul 2021   Prob (F-statistic):               0.00
Time:                        10:13:58   Log-Likelihood:                 1531.0
No. Observations:                1946   AIC:                            -3032.
Df Residuals:                    1931   BIC:                            -2948.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           1.4645      0.062     23.781      0.000       1.344       1.585
CROSS_SELL_SUCCESS                 -0.0127      0.005     -2.354      0.019      -0.023      -0.002
PRODUCT_CATEGORIES_VIEWED           0.0020      0.001      2.393      0.017       0.000       0.004
TASTES_AND_PREFERENCES              0.0120      0.006      2.142      0.032       0.001       0.023
MASTER_CLASSES_ATTENDED             0.0278      0.005      6.064      0.000       0.019       0.037
MEDIAN_MEAL_RATING                  0.0646      0.004     14.944      0.000       0.056       0.073
TOTAL_PHOTOS_VIEWED              5.516e-05   2.04e-05      2.707      0.007    1.52e-05    9.51e-05
yahoo                               0.0210      0.007      2.904      0.004       0.007       0.035
log_AVG_ORDER_SIZE                 -0.1455      0.026     -5.493      0.000      -0.197      -0.094
log_AVG_PREP_VID_TIME               0.5660      0.036     15.539      0.000       0.495       0.637
log_TOTAL_MEALS_ORDERED             0.2575      0.011     23.372      0.000       0.236       0.279
log_UNIQUE_MEALS_PURCH             -0.1647      0.009    -17.758      0.000      -0.183      -0.147
log_CONTACTS_W_CUSTOMER_SERVICE     0.1315      0.018      7.119      0.000       0.095       0.168
has_WEEKLY_PLAN                    -0.0126      0.006     -2.151      0.032      -0.024      -0.001
has_VIEWED_PHOTOS                   0.0200      0.007      2.740      0.006       0.006       0.034
==============================================================================
Omnibus:                      229.218   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              930.906
Skew:                          -0.513   Prob(JB):                    7.18e-203
Kurtosis:                       6.229   Cond. No.                     6.08e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# STEP 8-2,building a full model with REVENUE,

# blueprinting a model type
#lm_full = smf.ols(formula = """REVENUE ~ CROSS_SELL_SUCCESS +

#CONTACTS_W_CUSTOMER_SERVICE +
#AVG_PREP_VID_TIME +
#AVG_ORDER_SIZE +
#MASTER_CLASSES_ATTENDED +
#MEDIAN_MEAL_RATING +
#AVG_CLICKS_PER_VISIT +
#TOTAL_PHOTOS_VIEWED +
#unitedhealth +
#yahoo +
#log_TOTAL_MEALS_ORDERED +
#log_UNIQUE_MEALS_PURCH +
#log_AVG_CLICKS_PER_VISIT  """,
#data = chef)


# telling Python to run the data through the blueprint
#results_full = lm_full.fit()


# printing the results
#results_full.summary()

In [17]:
#df_corr = chef.corr().round(2)


# printing (Pearson) correlations with SalePrice
#print(df_corr.loc['log_REVENUE'].sort_values(ascending = False))

<span class="mark">PART II- MODEL TOURNAMENT</span>

In [18]:
# importing libraries, importing dataset
import pandas as pd # data science essentials
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # enhanced data visualization
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# specifying the path and file name
file = './chef_ready.xlsx'


# reading the file into Python
chef_ready = pd.read_excel(file)


# checking the file
#chef_ready.head(n = 5)

In [19]:
#preparing training and testing datasets

# preparing explanatory variable data by dropping "REVENUE", and "log_REVENUE"
chef_ready_data   = chef_ready.drop(['REVENUE',
                               'log_REVENUE'],
                                axis = 1)

# preparing response variable data
#chef_ready_target = chef_ready.loc[ : , 'REVENUE']

log_chef_ready_target = chef_ready.loc[ : , 'log_REVENUE'] # use log_REVENUE in my assignment


# preparing training and testing sets (all letters are lowercase)
x_train, x_test, y_train, y_test = train_test_split(
            chef_ready_data,
            log_chef_ready_target,
            test_size = 0.25,
            random_state = 219)


# checking the shapes of the datasets
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape}
""")


Training Data
-------------
X-side: (1459, 76)
y-side: (1459,)


Testing Data
------------
X-side: (487, 76)
y-side: (487,)



In [20]:
# declaring set of x-variables
x_variables=['CROSS_SELL_SUCCESS',
             'PRODUCT_CATEGORIES_VIEWED',
'TASTES_AND_PREFERENCES',
'MEDIAN_MEAL_RATING' ,
'TOTAL_PHOTOS_VIEWED' ,
'yahoo',
'log_AVG_PREP_VID_TIME',
'log_TOTAL_MEALS_ORDERED',
'AVG_ORDER_SIZE',
'log_UNIQUE_MEALS_PURCH',
'log_CONTACTS_W_CUSTOMER_SERVICE',
'has_MASTER_CLASSES_ATTENDED',
'has_WEEKLY_PLAN',
'walmart']
for val in x_variables:
    print(f"{val} +")
  


CROSS_SELL_SUCCESS +
PRODUCT_CATEGORIES_VIEWED +
TASTES_AND_PREFERENCES +
MEDIAN_MEAL_RATING +
TOTAL_PHOTOS_VIEWED +
yahoo +
log_AVG_PREP_VID_TIME +
log_TOTAL_MEALS_ORDERED +
AVG_ORDER_SIZE +
log_UNIQUE_MEALS_PURCH +
log_CONTACTS_W_CUSTOMER_SERVICE +
has_MASTER_CLASSES_ATTENDED +
has_WEEKLY_PLAN +
walmart +


In [21]:
# merging X_train and y_train so that they can be used in statsmodels
chef_ready_train = pd.concat([x_train, y_train], axis = 1)


# Step 1: build a model
lm_best = smf.ols(formula =  """log_REVENUE ~ CROSS_SELL_SUCCESS +
PRODUCT_CATEGORIES_VIEWED +
TASTES_AND_PREFERENCES +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
TOTAL_PHOTOS_VIEWED +
yahoo +
log_AVG_ORDER_SIZE +
log_AVG_PREP_VID_TIME +
log_TOTAL_MEALS_ORDERED +
log_UNIQUE_MEALS_PURCH +
log_CONTACTS_W_CUSTOMER_SERVICE +
has_WEEKLY_PLAN +
walmart """,
data =chef_ready_train)


# Step 2: fit the model based on the data
results = lm_best.fit()



# Step 3: analyze the summary output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            log_REVENUE   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     278.3
Date:                Mon, 05 Jul 2021   Prob (F-statistic):               0.00
Time:                        10:14:01   Log-Likelihood:                 1165.6
No. Observations:                1459   AIC:                            -2301.
Df Residuals:                    1444   BIC:                            -2222.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

<span class="girk">OLS MODEL</span>

In [22]:
# applying model in scikit-learn 

# Preparing a DataFrame based the the analysis above
ols_data   = chef_ready.loc[ : , x_variables]


# Preparing the target variable
log_chef_ready_target = chef_ready.loc[ : , 'log_REVENUE']


###############################################
## setting up more than one train-test split ##
###############################################
# FULL X-dataset (log Y)
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            chef_ready_data,     # x-variables
            log_chef_ready_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


# OLS p-value x-dataset (log Y)
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,         # x-variables
            log_chef_ready_target,   # y-variable
            test_size = 0.25,
            random_state = 219)

In [23]:
# INSTANTIATING a model object
lr = LinearRegression()


# FITTING to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
lr_pred = lr_fit.predict(x_test_OLS)


# SCORING the results
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4))
print('OLS Testing Score  :', lr.score(x_test_OLS, y_test_OLS).round(4))


# saving scoring data for future use
lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4) # using R-square
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

OLS Training Score : 0.7309
OLS Testing Score  : 0.7223
OLS Train-Test Gap : 0.0086


In [24]:
# zipping each feature name to its coefficient
lr_model_values = zip(chef_ready_data[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    

# checking the results
for pair in lr_model_lst:
    print(pair)

('intercept', 1.45)
('CROSS_SELL_SUCCESS', -0.02)
('PRODUCT_CATEGORIES_VIEWED', 0.0)
('TASTES_AND_PREFERENCES', 0.01)
('MEDIAN_MEAL_RATING', 0.06)
('TOTAL_PHOTOS_VIEWED', 0.0)
('yahoo', 0.03)
('log_AVG_PREP_VID_TIME', 0.58)
('log_TOTAL_MEALS_ORDERED', 0.25)
('AVG_ORDER_SIZE', -0.02)
('log_UNIQUE_MEALS_PURCH', -0.18)
('log_CONTACTS_W_CUSTOMER_SERVICE', 0.11)
('has_MASTER_CLASSES_ATTENDED', 0.04)
('has_WEEKLY_PLAN', -0.01)
('walmart', 0.07)


In [25]:
# zipping each feature name to its coefficient
lr_model_values = zip(chef_ready_data[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    

# checking the results
for pair in lr_model_lst:
    print(pair)

('intercept', 1.45)
('CROSS_SELL_SUCCESS', -0.02)
('PRODUCT_CATEGORIES_VIEWED', 0.0)
('TASTES_AND_PREFERENCES', 0.01)
('MEDIAN_MEAL_RATING', 0.06)
('TOTAL_PHOTOS_VIEWED', 0.0)
('yahoo', 0.03)
('log_AVG_PREP_VID_TIME', 0.58)
('log_TOTAL_MEALS_ORDERED', 0.25)
('AVG_ORDER_SIZE', -0.02)
('log_UNIQUE_MEALS_PURCH', -0.18)
('log_CONTACTS_W_CUSTOMER_SERVICE', 0.11)
('has_MASTER_CLASSES_ATTENDED', 0.04)
('has_WEEKLY_PLAN', -0.01)
('walmart', 0.07)


<span class="girk">LASSO MODEL</span>(<span class="burk">this is the final model</span>)

In [26]:

# INSTANTIATING a model object
import sklearn.linear_model # linear models
lasso_model = sklearn.linear_model.Lasso(alpha     = 0.0005,  # changed alpha to 0.0005
                                         normalize = False) # change magitude to false


# FITTING to the training data
lasso_fit = lasso_model.fit(x_train_FULL, y_train_FULL)


# PREDICTING on new data
lasso_pred = lasso_fit.predict(x_test_FULL)


# SCORING the results
print('Lasso Training Score :', lasso_model.score(x_train_FULL, y_train_FULL).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test_FULL, y_test_FULL).round(4))


## the following code has been provided for you ##

# saving scoring data for future use
lasso_train_score = lasso_model.score(x_train_FULL, y_train_FULL).round(4) # using R-square
lasso_test_score  = lasso_model.score(x_test_FULL, y_test_FULL).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)

Lasso Training Score : 0.7684
Lasso Testing Score  : 0.7648
Lasso Train-Test Gap : 0.0036


In [27]:
# zipping each feature name to its coefficient
lasso_model_values = zip(chef_ready_data.columns, lasso_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
lasso_model_lst = [('intercept', lasso_fit.intercept_.round(decimals = 5))]


# printing out each feature-coefficient pair one by one
for val in lasso_model_values:
    lasso_model_lst.append(val)
    

# checking the results
for pair in lasso_model_lst:
    print(pair)

('intercept', 2.43086)
('CROSS_SELL_SUCCESS', -0.01411)
('TOTAL_MEALS_ORDERED', -0.00072)
('UNIQUE_MEALS_PURCH', 0.04453)
('CONTACTS_W_CUSTOMER_SERVICE', 0.00851)
('PRODUCT_CATEGORIES_VIEWED', 0.00216)
('AVG_TIME_PER_SITE_VISIT', 1e-05)
('MOBILE_NUMBER', 0.00093)
('CANCELLATIONS_BEFORE_NOON', 0.00175)
('CANCELLATIONS_AFTER_NOON', -0.0008)
('TASTES_AND_PREFERENCES', 0.00819)
('PC_LOGINS', -0.00053)
('MOBILE_LOGINS', -0.00619)
('WEEKLY_PLAN', 0.00017)
('EARLY_DELIVERIES', -0.00122)
('LATE_DELIVERIES', -0.0005)
('PACKAGE_LOCKER', -0.00873)
('REFRIGERATED_LOCKER', -0.00494)
('AVG_PREP_VID_TIME', 0.00142)
('AVG_ORDER_SIZE', -0.0115)
('MASTER_CLASSES_ATTENDED', 0.00309)
('MEDIAN_MEAL_RATING', 0.05391)
('AVG_CLICKS_PER_VISIT', -0.00349)
('TOTAL_PHOTOS_VIEWED', 7e-05)
('EMAIL_type', -0.0)
('amex', 0.0)
('aol', 0.0)
('apple', -0.0)
('boeing', 0.0)
('caterpillar', 0.0)
('chevron', -0.0)
('cisco', 0.0)
('cocacola', 0.0)
('disney', -0.0)
('dupont', 0.0)
('exxon', 0.0)
('ge', -0.0)
('gmail', 0.0)
(

In [28]:
## This code may have to be run more than once ##

# dropping coefficients that are equal to zero

# printing out each feature-coefficient pair one by one
for feature, coefficient in lasso_model_lst:
        
        if coefficient == 0:
            lasso_model_lst.remove((feature, coefficient))

            
# checking the results
for pair in lasso_model_lst:
    print(pair)

('intercept', 2.43086)
('CROSS_SELL_SUCCESS', -0.01411)
('TOTAL_MEALS_ORDERED', -0.00072)
('UNIQUE_MEALS_PURCH', 0.04453)
('CONTACTS_W_CUSTOMER_SERVICE', 0.00851)
('PRODUCT_CATEGORIES_VIEWED', 0.00216)
('AVG_TIME_PER_SITE_VISIT', 1e-05)
('MOBILE_NUMBER', 0.00093)
('CANCELLATIONS_BEFORE_NOON', 0.00175)
('CANCELLATIONS_AFTER_NOON', -0.0008)
('TASTES_AND_PREFERENCES', 0.00819)
('PC_LOGINS', -0.00053)
('MOBILE_LOGINS', -0.00619)
('WEEKLY_PLAN', 0.00017)
('EARLY_DELIVERIES', -0.00122)
('LATE_DELIVERIES', -0.0005)
('PACKAGE_LOCKER', -0.00873)
('REFRIGERATED_LOCKER', -0.00494)
('AVG_PREP_VID_TIME', 0.00142)
('AVG_ORDER_SIZE', -0.0115)
('MASTER_CLASSES_ATTENDED', 0.00309)
('MEDIAN_MEAL_RATING', 0.05391)
('AVG_CLICKS_PER_VISIT', -0.00349)
('TOTAL_PHOTOS_VIEWED', 7e-05)
('amex', 0.0)
('apple', -0.0)
('caterpillar', 0.0)
('cisco', 0.0)
('disney', -0.0)
('exxon', 0.0)
('gmail', 0.0)
('homedepot', -0.0)
('ibm', 0.0)
('jnj', -0.0)
('live', 0.0)
('me', -0.0)
('microsoft', -0.0)
('msn', -0.0)
('passpo

<span class="girk">ARD MODEL</span>

In [29]:
#ARD MODEL
# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression(normalize  = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train_FULL, y_train_FULL)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test_FULL)


print('Training Score:', ard_model.score(x_train_FULL, y_train_FULL).round(4))
print('Testing Score :',  ard_model.score(x_test_FULL, y_test_FULL).round(4))


# saving scoring data for future use
ard_train_score = ard_model.score(x_train_FULL, y_train_FULL).round(4)
ard_test_score  = ard_model.score(x_test_FULL, y_test_FULL).round(4)


# displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

Training Score: 0.767
Testing Score : 0.7587
ARD Train-Test Gap : 0.0083


In [30]:
# zipping each feature name to its coefficient
ard_model_values = zip(chef_ready_data.columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)
    

# checking the results
for pair in ard_model_lst:
    print(pair)

('intercept', 1.89)
('CROSS_SELL_SUCCESS', -0.01223)
('TOTAL_MEALS_ORDERED', 0.0)
('UNIQUE_MEALS_PURCH', 0.0482)
('CONTACTS_W_CUSTOMER_SERVICE', 0.0)
('PRODUCT_CATEGORIES_VIEWED', 0.0)
('AVG_TIME_PER_SITE_VISIT', 0.0)
('MOBILE_NUMBER', 0.0)
('CANCELLATIONS_BEFORE_NOON', 0.0)
('CANCELLATIONS_AFTER_NOON', 0.0)
('TASTES_AND_PREFERENCES', 0.0)
('PC_LOGINS', 0.0)
('MOBILE_LOGINS', 0.0)
('WEEKLY_PLAN', 0.0)
('EARLY_DELIVERIES', 0.0)
('LATE_DELIVERIES', 0.0)
('PACKAGE_LOCKER', 0.0)
('REFRIGERATED_LOCKER', 0.0)
('AVG_PREP_VID_TIME', 0.0)
('AVG_ORDER_SIZE', -0.01556)
('MASTER_CLASSES_ATTENDED', 0.0)
('MEDIAN_MEAL_RATING', 0.05497)
('AVG_CLICKS_PER_VISIT', 0.01553)
('TOTAL_PHOTOS_VIEWED', 0.0)
('EMAIL_type', 0.0)
('amex', 0.0)
('aol', 0.0)
('apple', 0.0)
('boeing', 0.00567)
('caterpillar', 0.0)
('chevron', 0.0)
('cisco', 0.00491)
('cocacola', 0.0)
('disney', 0.0)
('dupont', 0.0)
('exxon', 0.0)
('ge', 0.0)
('gmail', 0.0)
('goldmansacs', 0.0)
('homedepot', 0.0)
('hotmail', 0.0)
('ibm', 0.0)
('inte

In [31]:
## This code may have to be run more than once ##

# dropping coefficients that are equal to zero

# printing out each feature-coefficient pair one by one
for feature, coefficient in ard_model_lst:
        
        if coefficient == 0:
            ard_model_lst.remove((feature, coefficient))

            
# checking the results
for pair in ard_model_lst:
    print(pair)

('intercept', 1.89)
('CROSS_SELL_SUCCESS', -0.01223)
('UNIQUE_MEALS_PURCH', 0.0482)
('PRODUCT_CATEGORIES_VIEWED', 0.0)
('MOBILE_NUMBER', 0.0)
('CANCELLATIONS_AFTER_NOON', 0.0)
('PC_LOGINS', 0.0)
('WEEKLY_PLAN', 0.0)
('LATE_DELIVERIES', 0.0)
('REFRIGERATED_LOCKER', 0.0)
('AVG_ORDER_SIZE', -0.01556)
('MEDIAN_MEAL_RATING', 0.05497)
('AVG_CLICKS_PER_VISIT', 0.01553)
('EMAIL_type', 0.0)
('aol', 0.0)
('boeing', 0.00567)
('chevron', 0.0)
('cisco', 0.00491)
('disney', 0.0)
('exxon', 0.0)
('gmail', 0.0)
('homedepot', 0.0)
('ibm', 0.0)
('jnj', 0.0)
('live', 0.0)
('me', 0.0)
('merck', -0.01789)
('microsoft', -0.01888)
('msn', 0.0)
('passport', 0.0)
('pg', 0.0)
('travelers', 0.0)
('unitedtech', 0.0)
('verizon', -0.04164)
('walmart', 0.03622)
('yahoo', 0.01757)
('log_AVG_TIME_PER_SITE_VISIT', 0.01021)
('log_AVG_PREP_VID_TIME', 0.5824)
('log_TOTAL_MEALS_ORDERED', 0.25829)
('log_UNIQUE_MEALS_PURCH', -0.59713)
('log_CONTACTS_W_CUSTOMER_SERVICE', 0.10465)
('log_AVG_CLICKS_PER_VISIT', -0.54967)
('has_WE

<span class="girk">Comparing results</span>

In [32]:
# comparing results

print(f"""
Model      Train Score      Test Score         Train-Test Gap        Model Size
-----      -----------      ----------         --------------        ----------
OLS        {lr_train_score}            {lr_test_score}               {lr_test_gap}                {len(lr_model_lst)}
Lasso      {lasso_train_score}            {lasso_test_score}               {lasso_test_gap}                {len(lasso_model_lst)}
ARD        {ard_train_score}             {ard_test_score}               {ard_test_gap}                {len(ard_model_lst)}
""")


# creating a dictionary for model results
model_performance = {
    
    'Model Type'    : ['OLS', 'Lasso', 'ARD'],
           
    'Training' : [lr_train_score, lasso_train_score,
                 ard_train_score],
           
    'Testing'  : [lr_test_score, lasso_test_score,
                ard_test_score],
                    
    'Train-Test Gap' : [lr_test_gap, lasso_test_gap,
                        ard_test_gap],
                    
    'Model Size' : [len(lr_model_lst), len(lasso_model_lst),
                    len(ard_model_lst)],
                    
    'Model' : [lr_model_lst, lasso_model_lst, ard_model_lst]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)


# sending model results to Excel
model_performance.to_excel('./log_revenue_chef_ready_linear_model_performance.xlsx',
                           index = False)


Model      Train Score      Test Score         Train-Test Gap        Model Size
-----      -----------      ----------         --------------        ----------
OLS        0.7309            0.7223               0.0086                15
Lasso      0.7684            0.7648               0.0036                53
ARD        0.767             0.7587               0.0083                46



In [33]:
prediction_results = pd.DataFrame(data = {
    'Original log_Revenue' : y_test_FULL,
    'LR Predictions'       : lr_pred.round(decimals = 5),
    'Lasso Predictions'    : lasso_pred.round(decimals = 5),
    'ARD Predictions'      : ard_pred.round(decimals = 5),
    'LR Deviation'         : lr_pred.round(decimals = 5) - y_test_FULL,
    'Lasso Deviation'      : lasso_pred.round(decimals = 5) - y_test_FULL,
    'ARD Deviation'        : ard_pred.round(decimals = 5) - y_test_FULL,
    })


prediction_results.to_excel(excel_writer = './log_revenue_chef_ready_linear_model_predictions.xlsx',
                            index = False)